In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from registration_utilities import *
from annotation_utilities import *
from metadata import *
from data_manager import *

Setting environment for Precision WorkStation


No vtk


In [7]:
def get_structure_contours_from_structure_volumes_v3(volumes, stack, sections, 
                                                     resolution, level, sample_every=1,
                                                    use_unsided_name_as_key=False):
    """
    Re-section atlas volumes and obtain structure contours on each section.
    Resolution of output contours are in volume resolution.
    v3 supports multiple levels.

    Args:
        volumes (dict of (3D array, 3-tuple)): {structure: (volume, origin_wrt_wholebrain)}. volume is a 3d array of probability values.
        sections (list of int):
        resolution (int): resolution of input volumes.
        level (float or dict or dict of list): the cut-off probability at which surfaces are generated from probabilistic volumes. Default is 0.5.
        sample_every (int): how sparse to sample contour vertices.

    Returns:
        Dict {section: {name_s: contour vertices}}.
    """

    from collections import defaultdict
    
    structure_contours_wrt_alignedBrainstemCrop_rawResol = defaultdict(lambda: defaultdict(dict))

    converter = CoordinatesConverter(stack=stack, section_list=metadata_cache['sections_to_filenames'][stack].keys())

    converter.register_new_resolution('structure_volume', resol_um=convert_resolution_string_to_um(resolution=resolution, stack=stack))
    converter.register_new_resolution('image', resol_um=convert_resolution_string_to_um(resolution='raw', stack=stack))
    
    for name_s, (structure_volume_volResol, origin_wrt_wholebrain_volResol) in volumes.iteritems():

        converter.derive_three_view_frames(name_s, 
        origin_wrt_wholebrain_um=convert_resolution_string_to_um(resolution=resolution, stack=stack) * origin_wrt_wholebrain_volResol,
        zdim_um=convert_resolution_string_to_um(resolution=resolution, stack=stack) * structure_volume_volResol.shape[2])

        positions_of_all_sections_wrt_structureVolume = converter.convert_frame_and_resolution(
        p=np.array(sections)[:,None],
        in_wrt=('wholebrain', 'sagittal'), in_resolution='section',
        out_wrt=(name_s, 'sagittal'), out_resolution='structure_volume')[..., 2].flatten()
            
        structure_ddim = structure_volume_volResol.shape[2]
        
        valid_mask = (positions_of_all_sections_wrt_structureVolume >= 0) & (positions_of_all_sections_wrt_structureVolume < structure_ddim)
        if np.count_nonzero(valid_mask) == 0:
#             sys.stderr.write("%s, valid_mask is empty.\n" % name_s)
            continue

        positions_of_all_sections_wrt_structureVolume = positions_of_all_sections_wrt_structureVolume[valid_mask]
        positions_of_all_sections_wrt_structureVolume = np.round(positions_of_all_sections_wrt_structureVolume).astype(np.int)
        
        if isinstance(level, dict):
            level_this_structure = level[name_s]
        else:
            level_this_structure = level

        if isinstance(level_this_structure, float):
            level_this_structure = [level_this_structure]
                        
        for one_level in level_this_structure:

            contour_2d_wrt_structureVolume_sectionPositions_volResol = \
            find_contour_points_3d(structure_volume_volResol >= one_level,
                                    along_direction='sagittal',
                                    sample_every=sample_every,
                                    positions=positions_of_all_sections_wrt_structureVolume)

            for d_wrt_structureVolume, cnt_uv_wrt_structureVolume in contour_2d_wrt_structureVolume_sectionPositions_volResol.iteritems():

                contour_3d_wrt_structureVolume_volResol = np.column_stack([cnt_uv_wrt_structureVolume, np.ones((len(cnt_uv_wrt_structureVolume),)) * d_wrt_structureVolume])

    #             contour_3d_wrt_wholebrain_uv_rawResol_section = converter.convert_frame_and_resolution(
    #                 p=contour_3d_wrt_structureVolume_volResol,
    #                 in_wrt=(name_s, 'sagittal'), in_resolution='structure_volume',
    #                 out_wrt=('wholebrain', 'sagittal'), out_resolution='image_image_section')

                contour_3d_wrt_alignedBrainstemCrop_uv_rawResol_section = converter.convert_frame_and_resolution(
                    p=contour_3d_wrt_structureVolume_volResol,
                    in_wrt=(name_s, 'sagittal'), in_resolution='structure_volume',
                    out_wrt=('wholebrainXYcropped', 'sagittal'), out_resolution='image_image_section')

                assert len(np.unique(contour_3d_wrt_alignedBrainstemCrop_uv_rawResol_section[:,2])) == 1
                sec = int(contour_3d_wrt_alignedBrainstemCrop_uv_rawResol_section[0,2])

                if use_unsided_name_as_key:
                    name = convert_to_unsided_label(name_s)
                else:
                    name = name_s

                structure_contours_wrt_alignedBrainstemCrop_rawResol[sec][name][one_level] = contour_3d_wrt_alignedBrainstemCrop_uv_rawResol_section[..., :2]
        
    return structure_contours_wrt_alignedBrainstemCrop_rawResol

# Generate manuscript figure

In [8]:
# CHATM2 section 106 shows 5N/7N/Amb, ROI: x=8112,y=8960,w=8608,h=6608
# CHATM2 section 162 shows 6N/10N/12N, ROI: x=10560,y=9552,w=8368,h=3072

In [9]:
# img = DataManager.load_image_v2(stack='CHATM2', prep_id=2, resol='raw', version='CHATJpeg', section=106)

version = 'NtbNormalizedAdaptiveInvertedGammaJpeg'
# version = 'CHATJpeg'
sec = 162
img = DataManager.load_image_v2(stack='CHATM2', prep_id=2, resol='raw', version=version, section=sec)
viz0 = gray2rgb(img)

Not using image_cache.


/data/CSHL_data_processed/CHATM2/CHATM2_prep2_raw_CHATJpeg/CHATM2_slide53_2018_01_22-S3_prep2_raw_CHATJpeg.jpg


In [10]:
for stack in ['CHATM2']:
    
    section_margin_um = 500.
    section_margin = int(section_margin_um / SECTION_THICKNESS)

    valid_secmin = np.min(metadata_cache['valid_sections'][stack])
    valid_secmax = np.max(metadata_cache['valid_sections'][stack])

    chat_structures_df = DataManager.load_annotation_v4(stack=stack, by_human=True, 
                                                   suffix='structuresHanddrawn', 
                                                   timestamp='latest')

    chat_structures_df = chat_structures_df[~chat_structures_df['volume'].isnull()]

    chat_structures, chat_structure_resolution = \
    convert_structure_annotation_to_volume_origin_dict_v2(structures_df=chat_structures_df, 
                                                          out_resolution='10.0um', stack=stack)

#     for structure_m in ['Amb_L', '5N_L', '7N_L']:
#     for structure_m in ['6N_L', '10N_L', '12N']:
    for structure_m in ['12N']:

        stack_m_spec = dict(name='atlasV6',
                           vol_type='score',
                           structure=structure_m,
                            resolution='10.0um'
                           )

        stack_f_spec = dict(name=stack,
                           vol_type='score',
                           detector_id=799,
                           structure=convert_to_original_name(structure_m),
                            resolution='10.0um'
                           )

        local_alignment_spec = dict(stack_m=stack_m_spec, 
                              stack_f=stack_f_spec,
                              warp_setting=27)

        vo = DataManager.load_transformed_volume_v2(alignment_spec=local_alignment_spec, return_origin_instead_of_bbox=True)

        registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners = \
        load_json('/home/yuncong/' + stack + '_registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners.json')

        (_, _, secmin), (_, _, secmax) = registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners[structure_m]

        atlas_structures_wrt_wholebrainWithMargin_sections = \
        range(max(secmin - section_margin, valid_secmin), min(secmax + 1 + section_margin, valid_secmax))

        levels = [0.1, 0.25, 0.5, 0.75, 0.99]

        contours_all_sections_all_structures_all_levels = \
        get_structure_contours_from_structure_volumes_v3(volumes={structure_m: vo}, stack=stack, 
                                                         sections=atlas_structures_wrt_wholebrainWithMargin_sections,
                                                        resolution='10.0um', level=levels, sample_every=5)

        ####################################

        chat_vo = chat_structures[structure_m]

        chat_contours_all_sections_all_structures_all_levels = \
        get_structure_contours_from_structure_volumes_v3(volumes={structure_m: chat_vo}, stack=stack, 
                                                         sections=atlas_structures_wrt_wholebrainWithMargin_sections,
                                                        resolution='10.0um', level=[.5], sample_every=1)

        ####################################

        # prep2 because at end of get_structure_contours_from_structure_volumes_v2 we used wholebrainXYcropped

#         for sec, contours_all_structures_all_levels in sorted(contours_all_sections_all_structures_all_levels.items()):
#         for sec, contours_all_structures_all_levels in [(106, contours_all_sections_all_structures_all_levels[106])]:

        contours_all_structures_all_levels = contours_all_sections_all_structures_all_levels[sec]

        if is_invalid(sec=sec, stack=stack):
            continue

#             for version in ['NtbNormalizedAdaptiveInvertedGammaJpeg', 'CHATJpeg']:
#             for version in ['CHATJpeg']:
#                 img = DataManager.load_image_v2(stack=stack, prep_id=2, resol='raw', version=version, section=sec)

        for name_s, cnt_all_levels in contours_all_structures_all_levels.iteritems():
#                     viz = gray2rgb(img)
#                     for level, cnt in cnt_all_levels.iteritems():
#                         cv2.polylines(viz, [cnt.astype(np.int)], isClosed=True, color=LEVEL_TO_COLOR_LINE[level], thickness=10)

#                     fp = os.path.join('/home/yuncong/' + stack + '_atlas_aligned_multilevel', name_s, version, stack + '_' + name_s + '_' + version + '_' + ('%03d' % sec) + '.jpg')
#                     print fp
#                     create_parent_dir_if_not_exists(fp)
#                     imsave(fp, viz)

#                     # Add CHAT contour
#                     if sec in chat_contours_all_sections_all_structures_all_levels:
#                         chat_cnt = chat_contours_all_sections_all_structures_all_levels[sec][name_s][.5]
#                         cv2.polylines(viz, [chat_cnt.astype(np.int)], isClosed=True, color=(255,255,255), thickness=20)

#                     fp = os.path.join('/home/yuncong/' + stack + '_atlas_aligned_multilevel_down16', name_s, version, stack + '_' + name_s + '_' + version + '_' + ('%03d' % sec) + '.jpg')
#                     print fp
#                     create_parent_dir_if_not_exists(fp)
#                     imsave(fp, viz[::16, ::16])

            #######################

            for level, cnt in cnt_all_levels.iteritems():
                cv2.polylines(viz0, [cnt.astype(np.int)], isClosed=True, color=LEVEL_TO_COLOR_LINE[level], thickness=10)

            if sec in chat_contours_all_sections_all_structures_all_levels:
                chat_cnt = chat_contours_all_sections_all_structures_all_levels[sec][name_s][.5]
                cv2.polylines(viz0, [chat_cnt.astype(np.int)], isClosed=True, color=(255,255,255), thickness=20)

                #######################



aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_labelings_v3/CHATM2" "/home/yuncong/CSHL_labelings_v3/CHATM2" --exclude "*" --include "*structuresHanddrawn*"
0
latest timestamp:  05122018234101


In [11]:
fp = os.path.join('/home/yuncong/' + stack + '_atlas_aligned_multilevel_all_structures', version, stack + '_' + version + '_' + ('%03d' % sec) + '.jpg')
print fp
create_parent_dir_if_not_exists(fp)
imsave(fp, viz0)

/home/yuncong/CHATM2_atlas_aligned_multilevel_all_structures/CHATJpeg/CHATM2_CHATJpeg_161.jpg


In [12]:
fp = os.path.join('/home/yuncong/' + stack + '_atlas_aligned_multilevel_down16_all_structures', version, stack + '_' + version + '_' + ('%03d' % sec) + '.jpg')
print fp
create_parent_dir_if_not_exists(fp)
imsave(fp, viz0[::16, ::16])

/home/yuncong/CHATM2_atlas_aligned_multilevel_down16_all_structures/CHATJpeg/CHATM2_CHATJpeg_161.jpg


# Summarize amount of deviation from CHAT

In [ ]:
deviation_from_chat_um_per_structure_all_stacks = {}

for stack in ['CHATM2', 'CHATM3']:
# for stack in ['CHATM2']:
    
#     stack = 'CHATM2' # 05092018200440
    # stack = 'CHATM3' # 05092018203614

    chat_structures_df = DataManager.load_annotation_v4(stack=stack, by_human=True, 
                                                   suffix='structuresHanddrawn', 
                                                   timestamp='latest')

    chat_structures_df = chat_structures_df[~chat_structures_df['volume'].isnull()]

    chat_structures, chat_structure_resolution = \
    convert_structure_annotation_to_volume_origin_dict_v2(structures_df=chat_structures_df, 
                                                          out_resolution='10.0um', stack=stack)

    deviation_from_chat_um_per_structure = {}

#     for name_s, (v_chat, o_chat) in sorted(chat_structures.items()):
    for name_s in ['5N_L', '5N_R', '6N_L', '6N_R', '7N_L', '7N_R', 'Amb_L', 'Amb_R', '10N_L', '10N_R', '12N']:
        
        (v_chat, o_chat) = chat_structures[name_s]
        
#         if name_s in ['3N_L', '3N_R', '4N_L', '4N_R']:
#             continue

        chat_centroid = get_centroid_3d(v_chat) + o_chat
#         chat_centroid = o_chat

        stack_m_spec = dict(name='atlasV6',
                       vol_type='score',
                       structure=name_s,
                        resolution='10.0um'
                       )

        stack_f_spec = dict(name=stack,
                           vol_type='score',
                           detector_id=799,
                           structure=convert_to_original_name(name_s),
                            resolution='10.0um'
                           )

        local_alignment_spec = dict(stack_m=stack_m_spec, 
                              stack_f=stack_f_spec,
                              warp_setting=27)

        v_alg, o_alg = DataManager.load_transformed_volume_v2(alignment_spec=local_alignment_spec, 
                                                      return_origin_instead_of_bbox=True)
        alg_centroid = get_centroid_3d(v_alg) + o_alg
#         alg_centroid = o_alg

#         print name_s, chat_centroid, alg_centroid, chat_centroid - alg_centroid, np.linalg.norm(chat_centroid - alg_centroid), 'x 10.0um'

#         deviation_from_chat_um_per_structure[name_s] = np.linalg.norm(chat_centroid - alg_centroid) * 10.
        deviation_from_chat_um_per_structure[name_s] = np.abs(chat_centroid[0] - alg_centroid[0]) * 10.

    deviation_from_chat_um_per_structure_all_stacks[stack] = deviation_from_chat_um_per_structure
    

fig, ax = plot_by_stack_by_structure(deviation_from_chat_um_per_structure_all_stacks, 
#                            structures=sorted(deviation_from_chat_um_per_structure.keys()), 
                           structures=['5N_L', '5N_R', '6N_L', '6N_R', '7N_L', '7N_R', 'Amb_L', 'Amb_R', '10N_L', '10N_R', '12N'],
                          yticks=np.arange(0, np.max(deviation_from_chat_um_per_structure.values())+10, 50),
                          ylabel='Error ($\mu$m)',
                                     xlabel='Structures (sorted from rostral to caudal)',
                        title='Rostral-caudal error of automated annotations compared to CHAT labels')

In [ ]:
bar_height = [std_rostral_caudal_all_structures[s] 
for s in ['5N_L', '5N_R', '6N_L', '6N_R', '7N_L', '7N_R', 'Amb_L', 'Amb_R', '10N_L', '10N_R', '12N']]

In [ ]:
ax.hlines(y=bar_height, xmin=np.arange(-.3, -.3+len(bar_height)), xmax=np.arange(.3, .3+len(bar_height)+1));

In [ ]:
import matplotlib.lines as mlines
black_line = mlines.Line2D([], [], color='black', label='Population std. var.')
handles, labels = ax.get_legend_handles_labels()
# ax.legend(handles, labels)
# import matplotlib.patches as mpatches
# red_patch = mpatches.Rectangle(xy=(0,0), width=1, height=1, angle=0.0, color='black', label='Population std var')
ax.legend(handles=handles + [black_line]);

In [ ]:
fig

In [5]:
import pandas as pd

In [10]:
pd.Series(rms_3d_all_structures).to_csv('/home/yuncong/population_rms_3d_all_structures.csv')

In [4]:
rms_3d_all_structures = \
{'10N_L': 131.71192054112927,
 '10N_R': 114.96417637822844,
 '12N': 91.3270700999804,
 '3N_L': 147.98449664547243,
 '3N_R': 152.16865163127156,
 '4N_L': 158.69662475344322,
 '4N_R': 158.43216221034416,
 '5N_L': 78.32631884362334,
 '5N_R': 85.95302152618588,
 '6N_L': 113.07629780975675,
 '6N_R': 218.17263584304976,
 '7N_L': 102.5725161699128,
 '7N_R': 72.63248091490208,
 '7n_L': 71.46592203158622,
 '7n_R': 75.25927329777348,
 'AP': 136.2815598245806,
 'Amb_L': 132.02746680538993,
 'Amb_R': 84.02518989273634,
 'DC_L': 155.3585723709082,
 'DC_R': 171.79567295576604,
 'IC': 181.10799688956945,
 'LC_L': 80.01409678709847,
 'LC_R': 98.99470628358642,
 'LRt_L': 179.27431940235496,
 'LRt_R': 161.8269305086202,
 'PBG_L': 238.33674289448834,
 'PBG_R': 250.83628545791547,
 'Pn_L': 94.42299470592697,
 'Pn_R': 121.40960765270191,
 'RMC_L': 74.15096911922737,
 'RMC_R': 98.74452140018367,
 'RtTg': 98.44958129019756,
 'SC': 152.07067811851826,
 'SNC_L': 176.72171558010933,
 'SNC_R': 376.7595848929037,
 'SNR_L': 135.78206710374852,
 'SNR_R': 189.01059052068905,
 'Sp5C_L': 182.64086075775114,
 'Sp5C_R': 215.61126983505886,
 'Sp5I_L': 137.720089625935,
 'Sp5I_R': 175.23520185721895,
 'Sp5O_L': 105.21523162672761,
 'Sp5O_R': 154.17620463439533,
 'Tz_L': 90.4634346426642,
 'Tz_R': 72.16627875545487,
 'VCA_L': 191.67689496954583,
 'VCA_R': 157.7867444645034,
 'VCP_L': 181.21763612969875,
 'VCP_R': 177.82775451903518,
 'VLL_L': 116.72853023806996,
 'VLL_R': 143.18241448763393}

In [1]:
# Got this from `update_atlas.ipynb`

std_rostral_caudal_all_structures = {'10N_L': 103.42021274923908,
 '10N_R': 75.95924780580737,
 '12N': 68.32049890900197,
 '3N_L': 54.97742349968691,
 '3N_R': 58.379548925477394,
 '4N_L': 30.67232267803452,
 '4N_R': 60.29220799807557,
 '5N_L': 56.57086576638428,
 '5N_R': 49.10816722833687,
 '6N_L': 74.3642772366564,
 '6N_R': 81.55248706408344,
 '7N_L': 48.31151635654887,
 '7N_R': 41.47692179437372,
 '7n_L': 35.19943253565955,
 '7n_R': 42.287104089026364,
 'AP': 88.0574920336851,
 'Amb_L': 73.53499710520586,
 'Amb_R': 40.78454054188801,
 'DC_L': 102.55766464685689,
 'DC_R': 84.42462719496737,
 'IC': 49.89508894942529,
 'LC_L': 54.742163166311165,
 'LC_R': 45.455029290349195,
 'LRt_L': 94.83452479180202,
 'LRt_R': 50.495272487349126,
 'PBG_L': 125.32155168375405,
 'PBG_R': 144.0799798618805,
 'Pn_L': 26.95352835751879,
 'Pn_R': 30.714566472150093,
 'RMC_L': 42.376941495891515,
 'RMC_R': 61.40273269428388,
 'RtTg': 33.81589003254259,
 'SC': 58.204642126226496,
 'SNC_L': 71.68790405871921,
 'SNC_R': 96.45458747105981,
 'SNR_L': 66.43441836299478,
 'SNR_R': 97.54493855224497,
 'Sp5C_L': 60.46272962021727,
 'Sp5C_R': 103.9787422763316,
 'Sp5I_L': 63.10318088419658,
 'Sp5I_R': 81.01362617645212,
 'Sp5O_L': 71.46808744368994,
 'Sp5O_R': 90.04113695199503,
 'Tz_L': 61.53237060781115,
 'Tz_R': 52.00457612796869,
 'VCA_L': 76.74775208219178,
 'VCA_R': 101.90966907267152,
 'VCP_L': 114.3421015638567,
 'VCP_R': 117.19213463683252,
 'VLL_L': 50.55337770750099,
 'VLL_R': 69.01056120588787}